
<div style="background: linear-gradient(90deg, #00B8D9, #00ACC1); padding: 40px 20px; border-radius: 15px; text-align: center; animation: fadeIn 2s ease-in-out;">
<h1 style="text-align:center; color:white; font-size:48px; margin-bottom: 10px">Supply Chain Analysis Dashboard</h1>
<h3 style="text-align:center; color:#ECEFF1; font-weight:normal; font-size: 20px;">Exploring logistics, revenue, and performance metrics</h3>
</div>

<style>
@keyframes fadeIn {
  from { opacity: 0; transform: translateY(-10px); }
  to { opacity: 1; transform: translateY(0); }
}
</style>


In [31]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import numpy as np

# Custom JSON serializer for NumPy arrays
def np_serializer(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

# Load dataset
df = pd.read_csv('data/SCA.csv')

# Define consistent colors
color_map = {'haircare': '#ADD8E6', 'skincare': '#90EE90', 'cosmetics': '#FFA500'}

# Preprocess data
df['Inventory_Turnover_Ratio'] = df['Number_of_products_sold'] / df['Stock_levels']

# --- Product Performance ---
sales_by_product = df.groupby('Product_type')['Number_of_products_sold'].sum().reset_index()
fig1 = px.pie(sales_by_product, values='Number_of_products_sold', names='Product_type', 
              title='Sales Distribution Across Product Categories', color_discrete_map=color_map, hole=0.4)

product_summary = df.groupby(['Product_type', 'SKU']).agg({'Revenue_generated': 'sum', 'Number_of_products_sold': 'sum'}).reset_index()
fig2 = px.scatter(product_summary, x='Number_of_products_sold', y='Revenue_generated', color='Product_type', 
                  hover_data=['SKU'], title='Revenue vs Sales Performance by Product Category', color_discrete_map=color_map)

# --- Customer Demographics ---
revenue_by_demo = df.groupby('Customer_demographics')['Revenue_generated'].sum().reset_index()
fig3 = px.pie(revenue_by_demo, values='Revenue_generated', names='Customer_demographics', 
              title='Revenue Contribution by Customer Segments', hole=0.4, color_discrete_sequence=px.colors.qualitative.Pastel)

sales_by_demo_product = df.groupby(['Product_type', 'Customer_demographics'])['Number_of_products_sold'].sum().reset_index()
fig4 = px.bar(sales_by_demo_product, x='Customer_demographics', y='Number_of_products_sold', color='Product_type', 
              barmode='group', title='Product Sales Breakdown by Customer Segments', color_discrete_map=color_map)

# --- Analyzing SKUs ---
'''
sku_revenue = df.groupby(['SKU', 'Product_type'])['Revenue_generated'].sum().reset_index()
fig5 = px.bar(sku_revenue, x='SKU', y='Revenue_generated', color='Product_type', 
              title='Revenue Generated per SKU by Product Category', color_discrete_map=color_map)

sku_orders = df.groupby(['SKU', 'Product_type'])['Number_of_products_sold'].sum().reset_index()
fig6 = px.bar(sku_orders, x='SKU', y='Number_of_products_sold', color='Product_type', 
              title='Order Quantities per SKU by Product Category', color_discrete_map=color_map)
'''

sku_revenue = df.groupby(['SKU', 'Product_type'])['Revenue_generated'].sum().reset_index()
fig5 = px.bar(sku_revenue, x='SKU', y='Revenue_generated', color='Product_type', 
              title='Revenue Generated per SKU by Product Category', color_discrete_map=color_map)

sku_orders = df.groupby(['SKU', 'Product_type'])['Number_of_products_sold'].sum().reset_index()
fig6 = px.bar(sku_orders, x='SKU', y='Number_of_products_sold', color='Product_type', 
              title='Order Quantities per SKU by Product Category', color_discrete_map=color_map)

# --- Cost Analysis ---
transport_costs = df.groupby('Transportation_modes')['Shipping_costs'].sum().reset_index()
fig7 = px.pie(transport_costs, values='Shipping_costs', names='Transportation_modes', 
              title='Transportation Cost Breakdown by Mode', hole=0.4)

carrier_costs = df.groupby('Shipping_carriers')['Shipping_costs'].mean().reset_index()
fig8 = px.bar(carrier_costs, x='Shipping_carriers', y='Shipping_costs', 
              title='Average Shipping Costs Across Carriers', color_discrete_sequence=['#4682B4'])

inventory_turnover = df[['SKU', 'Product_type', 'Inventory_Turnover_Ratio']].sort_values('Inventory_Turnover_Ratio', ascending=False)
fig9 = px.bar(inventory_turnover, x='SKU', y='Inventory_Turnover_Ratio', color='Product_type', 
              title='Inventory Turnover Efficiency by SKU', color_discrete_map=color_map)

''' 
inventory_turnover = df[['SKU', 'Product_type', 'Inventory_Turnover_Ratio']].sort_values('Inventory_Turnover_Ratio', ascending=False)
fig9 = px.bar(inventory_turnover, x='SKU', y='Inventory_Turnover_Ratio', color='Product_type', 
              title='Inventory Turnover Efficiency by SKU', color_discrete_map=color_map)
''' 
# --- Supplier Analysis ---
supplier_demand = df.groupby('Supplier_name')['Number_of_products_sold'].sum().reset_index().sort_values('Number_of_products_sold', ascending=False)
fig10 = px.bar(supplier_demand, x='Supplier_name', y='Number_of_products_sold', 
               title='Top Suppliers by Product Demand', color_discrete_sequence=['#32CD32'])

supplier_defects = df.groupby('Supplier_name')['Defect_rates'].mean().reset_index().sort_values('Defect_rates', ascending=False)
fig11 = px.bar(supplier_defects, x='Supplier_name', y='Defect_rates', color='Supplier_name', 
               title='Supplier Quality: Average Defect Rates', color_discrete_sequence=px.colors.qualitative.Set2)

# --- Quality Control ---
transport_defects = df.groupby('Transportation_modes')['Defect_rates'].mean().reset_index()
fig12 = px.pie(transport_defects, values='Defect_rates', names='Transportation_modes', 
               title='Quality Impact by Transportation Mode', hole=0.4)

carrier_defects = df.groupby('Shipping_carriers')['Defect_rates'].mean().reset_index()
fig13 = px.bar(carrier_defects, x='Shipping_carriers', y='Defect_rates', 
               title='Quality Impact by Shipping Carrier', color_discrete_sequence=['#FF6347'])

product_defects = df.groupby('Product_type')['Defect_rates'].mean().reset_index()
fig14 = px.pie(product_defects, values='Defect_rates', names='Product_type', 
               title='Quality Impact by Product Category', color_discrete_map=color_map, hole=0.4)

# HTML Template with Centered fig13
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SCA Insights Dashboard</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: 'Segoe UI', sans-serif;
            background-color: #f0f2f5;
            margin: 0;
            padding: 20px;
        }}
        .dashboard-container {{
            max-width: 1400px;
            margin: auto;
            background: #ffffff;
            border-radius: 12px;
            box-shadow: 0 6px 12px rgba(0,0,0,0.1);
            padding: 25px;
        }}
        .header {{
            text-align: center;
            background: linear-gradient(90deg, #1e3c72, #2a5298);
            color: white;
            padding: 25px;
            border-radius: 12px 12px 0 0;
            margin-bottom: 20px;
        }}
        .header h1 {{
            margin: 0;
            font-size: 28px;
        }}
        .header p {{
            margin: 5px 0 0;
            font-size: 14px;
            opacity: 0.9;
        }}
        .tabs {{
            display: flex;
            justify-content: space-around;
            border-bottom: 2px solid #e0e0e0;
            margin-bottom: 20px;
        }}
        .tablink {{
            background-color: #2c3e50;
            color: white;
            border: none;
            padding: 12px 20px;
            cursor: pointer;
            transition: background-color 0.3s, transform 0.2s;
            flex-grow: 1;
            text-align: center;
            font-size: 14px;
        }}
        .tablink:hover {{
            background-color: #3498db;
            transform: scale(1.05);
        }}
        .tablink.active {{
            background-color: #2980b9;
        }}
        .tabcontent {{
            display: none;
        }}
        .global-filters, .individual-filters {{
            background: #ecf0f1;
            padding: 15px;
            border-radius: 8px;
            margin-bottom: 20px;
            display: flex;
            gap: 20px;
            align-items: center;
        }}
        .chart-container {{
            margin-bottom: 30px;
        }}
        .pie-row {{
            display: flex;
            justify-content: space-between;
            gap: 20px;
            margin-bottom: 30px;
        }}
        .pie-chart {{
            flex: 1;
            min-width: 0;
        }}
        select, input[type="range"] {{
            padding: 5px;
            border-radius: 4px;
            border: 1px solid #ccc;
        }}
        #fig_ctr-container {{
            display: flex;
            justify-content: center;
            margin-bottom: 30px;
        }}
        #fig13 {{
            width: 50%; /* Centered with fixed width */
        }}
    </style>
</head>
<body>
    <div class="dashboard-container">
        <div class="header">
            <h1>Supply Chain Insights Dashboard</h1>
            <p>Actionable Analytics for Optimization | Powered by xAI</p>
        </div>

        <!-- Global Filters -->
        <div class="global-filters">
            <label>Product Type: </label>
            <select id="globalProductFilter" onchange="updateCharts()">
                <option value="all">All</option>
                <option value="haircare">Haircare</option>
                <option value="skincare">Skincare</option>
                <option value="cosmetics">Cosmetics</option>
            </select>
        </div>

        <!-- Tabs -->
        <div class="tabs">
            <button class="tablink" onclick="openTab(event, 'Product')">Product Performance</button>
            <button class="tablink" onclick="openTab(event, 'Demographics')">Customer Insights</button>
            <button class="tablink" onclick="openTab(event, 'SKUs')">SKU Analysis</button>
            <button class="tablink" onclick="openTab(event, 'Costs')">Cost Insights</button>
            <button class="tablink" onclick="openTab(event, 'Suppliers')">Supplier Performance</button>
            <button class="tablink" onclick="openTab(event, 'Quality')">Quality Metrics</button>
        </div>

        <!-- Product Performance Tab -->
        <div id="Product" class="tabcontent">
        
            <div class="chart-container" id="fig1"></div>
            
            <div class="individual-filters">
                <label>Filter Revenue vs Sales by Product Type: </label>
                <select id="productFilter" onchange="updateCharts()">
                    <option value="all">All</option>
                    <option value="haircare">Haircare</option>
                    <option value="skincare">Skincare</option>
                    <option value="cosmetics">Cosmetics</option>
                </select>
            </div>
            
            <div class="chart-container" id="fig2"></div>
        </div>

        <!-- Customer Demographics Tab -->
        <div id="Demographics" class="tabcontent">
            <div class="pie-row">
                <div class="chart-container" id="fig3"></div>
                <div class="chart-container" id="fig4"></div>
            </div>
        </div>

        <!-- SKU Analysis Tab -->
        <div id="SKUs" class="tabcontent">
            <div class="pie-row">
                <div class="chart-container" id="fig5"></div>
                <div class="chart-container" id="fig6"></div>
            </div>
        </div>

        <!-- Cost Analysis Tab -->
        <div id="Costs" class="tabcontent">
            <div class="pie-row">
                <div class="pie-chart" id="fig7"></div>
                <div class="pie-chart" id="fig8"></div>
            </div>
            <div id="fig_ctr-container">
                <div id="fig9"></div>
            </div>
        </div>

        <!-- Supplier Analysis Tab -->
        <div id="Suppliers" class="tabcontent">
            <div class="pie-row">
                <div class="chart-container" id="fig10"></div>
                <div class="chart-container" id="fig11"></div>
            </div>
        </div>

        <!-- Quality Control Tab -->
        <div id="Quality" class="tabcontent">
            <div class="pie-row">
                <div class="pie-chart" id="fig12"></div>
                <div class="pie-chart" id="fig14"></div>
            </div>
            <div id="fig_ctr-container">
                <div id="fig13"></div>
            </div>
        </div>
    </div>

    <script>
        function openTab(evt, tabName) {{
            var i, tabcontent, tablinks;
            tabcontent = document.getElementsByClassName("tabcontent");
            for (i = 0; i < tabcontent.length; i++) {{
                tabcontent[i].style.display = "none";
            }}
            tablinks = document.getElementsByClassName("tablink");
            for (i = 0; i < tablinks.length; i++) {{
                tablinks[i].className = tablinks[i].className.replace(" active", "");
            }}
            document.getElementById(tabName).style.display = "block";
            evt.currentTarget.className += " active";
        }}

        // Default tab
        document.getElementsByClassName("tablink")[0].click();

        // Chart data
        var figures = {{
            'fig1': {json.dumps(fig1.to_dict(), default=np_serializer)},
            'fig2': {json.dumps(fig2.to_dict(), default=np_serializer)},
            'fig3': {json.dumps(fig3.to_dict(), default=np_serializer)},
            'fig4': {json.dumps(fig4.to_dict(), default=np_serializer)},
            'fig5': {json.dumps(fig5.to_dict(), default=np_serializer)},
            'fig6': {json.dumps(fig6.to_dict(), default=np_serializer)},
            'fig7': {json.dumps(fig7.to_dict(), default=np_serializer)},
            'fig8': {json.dumps(fig8.to_dict(), default=np_serializer)},
            'fig9': {json.dumps(fig9.to_dict(), default=np_serializer)},
            'fig10': {json.dumps(fig10.to_dict(), default=np_serializer)},
            'fig11': {json.dumps(fig11.to_dict(), default=np_serializer)},
            'fig12': {json.dumps(fig12.to_dict(), default=np_serializer)},
            'fig13': {json.dumps(fig13.to_dict(), default=np_serializer)},
            'fig14': {json.dumps(fig14.to_dict(), default=np_serializer)}
        }};

        function updateCharts() {{
            var globalProductFilter = document.getElementById('globalProductFilter').value;
            var productFilter = document.getElementById('productFilter') ? document.getElementById('productFilter').value : 'all';

            // Apply global filter to all charts
            function applyGlobalFilter(fig) {{
                if (globalProductFilter !== 'all') {{
                    return fig.data.filter(d => !d.name || d.name === globalProductFilter);
                }}
                return fig.data;
            }}

            // Product Tab
            var filteredFig2 = JSON.parse(JSON.stringify(figures['fig2']));
            if (productFilter !== 'all') {{
                filteredFig2.data = filteredFig2.data.filter(d => d.name === productFilter);
            }} else {{
                filteredFig2.data = applyGlobalFilter(figures['fig2']);
            }}
            Plotly.newPlot('fig1', applyGlobalFilter(figures['fig1']), figures['fig1'].layout);
            Plotly.newPlot('fig2', filteredFig2.data, figures['fig2'].layout);

            // Demographics Tab
            Plotly.newPlot('fig3', applyGlobalFilter(figures['fig3']), figures['fig3'].layout);
            Plotly.newPlot('fig4', applyGlobalFilter(figures['fig4']), figures['fig4'].layout);

            // SKU Analysis Tab
            Plotly.newPlot('fig5', applyGlobalFilter(figures['fig5']), figures['fig5'].layout);
            Plotly.newPlot('fig6', applyGlobalFilter(figures['fig6']), figures['fig6'].layout);

            // Cost Analysis Tab
            Plotly.newPlot('fig7', figures['fig7'].data, figures['fig7'].layout);
            Plotly.newPlot('fig8', figures['fig8'].data, figures['fig8'].layout);
            Plotly.newPlot('fig9', applyGlobalFilter(figures['fig9']), figures['fig9'].layout);

            // Supplier Analysis Tab
            Plotly.newPlot('fig10', figures['fig10'].data, figures['fig10'].layout);
            Plotly.newPlot('fig11', figures['fig11'].data, figures['fig11'].layout);

            // Quality Control Tab
            Plotly.newPlot('fig12', figures['fig12'].data, figures['fig12'].layout);
            Plotly.newPlot('fig13', figures['fig13'].data, figures['fig13'].layout);
            Plotly.newPlot('fig14', applyGlobalFilter(figures['fig14']), figures['fig14'].layout);
        }}

        // Initial plot
        updateCharts();
    </script>
</body>
</html>
"""

# Write to file
with open("SCA_Dashboard.html", "w") as f:
    f.write(html_content)

print("Enhanced Dashboard generated as 'SCA_Dashboard.html'")

Enhanced Dashboard generated as 'SCA_Dashboard.html'
